In [1]:
!pip install pyyaml h5py


[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 73.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [3]:
import os
import tensorflow as tf
from tensorflow import keras

In [4]:
(train_images,train_labels), (test_images,test_labels) = keras.datasets.mnist.load_data()

print(f"trening -> {train_images.shape}")
print(f"testowanie -> {test_images.shape}")

11490434/11490434 [==============================] - 1s 0us/step
trening -> (60000, 28, 28)
testowanie -> (10000, 28, 28)


In [5]:
train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1,28*28)/255.0
test_images = test_images[:1000].reshape(-1,28*28)/255.0

In [6]:
train_images.shape

(1000, 784)

In [7]:
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512,activation='relu',input_shape=(784,)),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(10)
    ])
    model.compile(optimizer='adam',
                  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=[tf.metrics.SparseCategoricalAccuracy()])
    return model

In [8]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [9]:
#użycie wywołania zwrotnego w punkcie kontrolnym
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                              save_weights_only=True,
                                              verbose=1)

model.fit(train_images,
          train_labels,
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])

Epoch 1/10
24/32 [=====================>........] - ETA: 0s - loss: 1.3192 - sparse_categorical_accuracy: 0.6328
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 2s 44ms/step - loss: 1.1648 - sparse_categorical_accuracy: 0.6770 - val_loss: 0.7218 - val_sparse_categorical_accuracy: 0.7790
Epoch 2/10
30/32 [===========================>..] - ETA: 0s - loss: 0.4346 - sparse_categorical_accuracy: 0.8771
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 32ms/step - loss: 0.4355 - sparse_categorical_accuracy: 0.8750 - val_loss: 0.5541 - val_sparse_categorical_accuracy: 0.8340
Epoch 3/10
26/32 [=======================>......] - ETA: 0s - loss: 0.3434 - sparse_categorical_accuracy: 0.9123
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 27ms/step - loss: 0.3214 - sparse_categorical_accuracy: 0.9180 - val_loss: 0.4936 - val_sparse_categorical_accuracy: 0.8420
Epoch 4/10
30/32 [=======

In [10]:
os.listdir(checkpoint_dir)

['checkpoint', 'cp.ckpt.data-00000-of-00001', 'cp.ckpt.index']

In [11]:
model = create_model()

loss,acc = model.evaluate(test_images,test_labels,verbose=2)
print(f"Niewytrenowany model: {acc}")

32/32 - 0s - loss: 2.3387 - sparse_categorical_accuracy: 0.0790 - 204ms/epoch - 6ms/step
Niewytrenowany model: 0.07900000363588333


In [12]:
model.load_weights(checkpoint_path)

loss,acc = model.evaluate(test_images,test_labels,verbose=2)
print(f"model po załadowaniu wag: {acc}")

32/32 - 0s - loss: 0.3945 - sparse_categorical_accuracy: 0.8710 - 95ms/epoch - 3ms/step
model po załadowaniu wag: 0.8709999918937683


In [13]:
#wywołanie zwrotne z opcją oddzwaniania do punktu kontrolnego
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
batch_size=32

In [14]:
cp_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*batch_size
)

In [15]:
model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))

model.fit(train_images,
          train_labels,
          epochs=50,
          batch_size=batch_size,
          callbacks=[cp_callback],
          validation_data=(test_images,test_labels),
          verbose=0)


Epoch 5: saving model to training_2/cp-0005.ckpt

Epoch 10: saving model to training_2/cp-0010.ckpt

Epoch 15: saving model to training_2/cp-0015.ckpt

Epoch 20: saving model to training_2/cp-0020.ckpt

Epoch 25: saving model to training_2/cp-0025.ckpt

Epoch 30: saving model to training_2/cp-0030.ckpt

Epoch 35: saving model to training_2/cp-0035.ckpt

Epoch 40: saving model to training_2/cp-0040.ckpt

Epoch 45: saving model to training_2/cp-0045.ckpt

Epoch 50: saving model to training_2/cp-0050.ckpt


In [16]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [17]:
model = create_model()
model.load_weights(latest)

loss,acc = model.evaluate(test_images,test_labels,verbose=2)
print(f"model po załadowaniu wag(latest): {acc}")

32/32 - 0s - loss: 0.4541 - sparse_categorical_accuracy: 0.8800 - 198ms/epoch - 6ms/step
model po załadowaniu wag(latest): 0.8799999952316284


In [18]:
model = create_model()
model.fit(train_images,train_labels,epochs=5)

!mkdir -p saved_model
model.save('saved_model/mojmodel')

Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 1.1942 - sparse_categorical_accuracy: 0.6600
Epoch 2/5
32/32 [==============================] - 0s 5ms/step - loss: 0.4561 - sparse_categorical_accuracy: 0.8690
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.3017 - sparse_categorical_accuracy: 0.9180
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2247 - sparse_categorical_accuracy: 0.9480
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1916 - sparse_categorical_accuracy: 0.9510
INFO:tensorflow:Assets written to: saved_model/mojmodel/assets


In [19]:
new_model = keras.models.load_model('saved_model/mojmodel')
new_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 512)               401920    
                                                                 
 dropout_4 (Dropout)         (None, 512)               0         
                                                                 
 dense_9 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [21]:
loss,acc = new_model.evaluate(test_images,test_labels,verbose=2)
print(f"model po załadowaniu wag(latest): {acc}")

32/32 - 0s - loss: 0.4088 - sparse_categorical_accuracy: 0.8710 - 231ms/epoch - 7ms/step
model po załadowaniu wag(latest): 0.8709999918937683


In [22]:
model = create_model()
model.fit(train_images,train_labels,epochs=5)

model.save('h5model.h5')

Epoch 1/5
32/32 [==============================] - 1s 5ms/step - loss: 1.1891 - sparse_categorical_accuracy: 0.6590
Epoch 2/5
32/32 [==============================] - 0s 6ms/step - loss: 0.4460 - sparse_categorical_accuracy: 0.8780
Epoch 3/5
32/32 [==============================] - 0s 5ms/step - loss: 0.3151 - sparse_categorical_accuracy: 0.9150
Epoch 4/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2397 - sparse_categorical_accuracy: 0.9340
Epoch 5/5
32/32 [==============================] - 0s 5ms/step - loss: 0.1718 - sparse_categorical_accuracy: 0.9600


In [23]:
hdf5_model = keras.models.load_model('h5model.h5')
hdf5_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 512)               401920    
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [24]:
loss,acc = hdf5_model.evaluate(test_images,test_labels,verbose=2)
print(f"model po załadowaniu wag(latest): {acc}")

32/32 - 0s - loss: 0.4100 - sparse_categorical_accuracy: 0.8710 - 174ms/epoch - 5ms/step
model po załadowaniu wag(latest): 0.8709999918937683
